In [ ]:
import pandas as pd
import numpy as np

In [ ]:

# Reload the data to fix the overwrite issue
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Merge the DataFrames
merged_movies = movies.merge(credits, on='title')

# Check the shape without overwriting `movies`
print(merged_movies.shape)  # Prints the shape of the merged DataFrame


In [ ]:
merged_movies.head()

In [ ]:
merged_movies = merged_movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
merged_movies.head()

In [ ]:
merged_movies.isnull().sum()

In [ ]:
merged_movies.dropna(inplace=True)

In [ ]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj): 
        L.append(i['name'])
    return L

In [ ]:
newgenre = merged_movies['genres'].apply(convert)

In [ ]:
newgenre.head()

In [ ]:
merged_movies['genres'] = newgenre

In [ ]:
merged_movies.head()

In [ ]:
newkeyword = merged_movies['keywords'].apply(convert)

In [ ]:
merged_movies['keywords'] = newkeyword

In [ ]:
def convertCastTop3(obj):
    L =[]
    counter = 0;
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
newCast = merged_movies['cast'].apply(convertCastTop3)

In [ ]:
merged_movies['cast'] = newCast

In [ ]:
def fetchDirector(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
newCrew = merged_movies['crew'].apply(fetchDirector)

In [ ]:
merged_movies['crew'] = newCrew

In [ ]:
merged_movies['overview'][0]

In [ ]:
newOverview = merged_movies['overview'].apply(lambda x:x.split())

In [ ]:
merged_movies['overview'] = newOverview

In [ ]:
merged_movies.head()

In [ ]:
merged_movies['genres'] = newgenre.apply(lambda x:[i.replace(" ","")for i in x])
merged_movies['cast']= newCast.apply(lambda x:[i.replace(" ","")for i in x])
merged_movies['crew']= newCrew.apply(lambda x:[i.replace(" ","")for i in x])
merged_movies['keywords']= newkeyword.apply(lambda x:[i.replace(" ","")for i in x])


In [ ]:
merged_movies.head()

In [ ]:
merged_movies['tags']= merged_movies['overview']+ merged_movies['genres'] + merged_movies['keywords']+ merged_movies['cast'] + merged_movies['crew']

In [ ]:
merged_movies.head()

In [ ]:
new_df = merged_movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'][0]

In [ ]:
# Clean each element and join the list into a single string
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(word.strip(' ,.') for word in x))


In [ ]:
new_df


In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df

In [ ]:
new_df['tags'][0]

In [ ]:
import sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words ="english")

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
print(cv.get_feature_names_out())  # List of all words in the vocabulary


In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y =[]

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
stem('in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'
    )

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df #4806 movies and 5000 words and each movie is a vector
#find distance btween two vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity # every movie is similar to itself diagonal is always 1

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances= similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x [1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Iron Man')

In [ ]:
recommend('The Avengers')

In [ ]:
recommend( "Harry Potter and the Sorcerer's Stone")

In [ ]:
import pickle

In [ ]:
new_df['title'].values

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl', 'wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))